# 获取拇指指尖与食指之间的距离-摄像头

同济子豪兄 2023-4-14

预备知识：https://www.bilibili.com/video/BV1x44y127Yu

## 导入工具包，定义可视化辅助函数

In [1]:
import cv2
import numpy as np
from PIL import Image
import mediapipe as mp

# 导入python绘图matplotlib
import matplotlib.pyplot as plt
# 使用ipython的魔法方法，将绘制出的图像直接嵌入在notebook单元格中
%matplotlib inline

## 导入模型

In [2]:
# 导入solution
mp_hands = mp.solutions.hands

# 导入模型
hands = mp_hands.Hands(static_image_mode=True,        # 是静态图片还是连续视频帧
                       max_num_hands=4,                # 最多检测几只手
                       min_detection_confidence=0.4,   # 置信度阈值，过滤低于该阈值的预测结果
                       min_tracking_confidence=0.5)    # 追踪阈值

# 导入绘图函数
mpDraw = mp.solutions.drawing_utils 

In [3]:
scaler = 1 # 字体大小因子

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## 逐帧处理函数

In [37]:
import pyautogui

print(pyautogui.size())


# 逐帧处理函数，默认不进行任何处理，直接将摄像头捕获的画面写入视频帧
def process_frame(img_bgr):
    
    bh, bw = pyautogui.size()
    
    
    
    # 获取图像宽高
    h, w = img_bgr.shape[0], img_bgr.shape[1]
    
    # 水平镜像翻转图像，使图中左右手与真实左右手对应
    # 参数 1：水平翻转，0：竖直翻转，-1：水平和竖直都翻转
    img_bgr = cv2.flip(img_bgr, 1)
    
    # BGR 转 RGB
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    
    # 将RGB图像输入模型，获取预测结果
    results = hands.process(img_rgb)
    # print('检测到 {} 只手'.format(len(results.multi_hand_landmarks)))
    
    if results.multi_hand_landmarks: # 如果有检测到手
    
        # 获取索引为hand_idx的手
        hand_idx = 0
        hand_21 = results.multi_hand_landmarks[hand_idx]

        # 可视化关键点及骨架连线
        mpDraw.draw_landmarks(img_bgr, hand_21, mp_hands.HAND_CONNECTIONS)
        
#         print(hand_21.landmark[4])
        # 获取食指指尖坐标
        index_tip_x = int(hand_21.landmark[4].x * w)
        index_tip_y = int(hand_21.landmark[4].y * h)
#         print(index_tip_x, index_tip_y)
        # 绘制圆：图像，圆心坐标，半径，BGR颜色，最后一个参数为线宽，-1表示填充
        img_bgr = cv2.circle(img_bgr,(index_tip_x, index_tip_y), 10, (255,240,1), -1)

        # 获取大拇指指尖坐标
        thumb_tip_x = int(hand_21.landmark[8].x * w)
        thumb_tip_y = int(hand_21.landmark[8].y * h)
        # 绘制圆：图像，圆心坐标，半径，BGR颜色，最后一个参数为线宽，-1表示填充
        img_bgr = cv2.circle(img_bgr, (thumb_tip_x, thumb_tip_y), 10, (94,218,121), -1)
        
        srceen_mid_mcp_x = int(hand_21.landmark[4].x * bw)
        srceen_mid_mcp_y = int(hand_21.landmark[4].y * bw)
        
        pyautogui.moveTo(srceen_mid_mcp_x, srceen_mid_mcp_y, duration=0)
    
        # 连接大拇指指尖和食指指尖
        # 图，两个点的坐标，颜色，线宽
        img_bgr = cv2.line(img_bgr, (index_tip_x,index_tip_y), (thumb_tip_x,thumb_tip_y), color=(1,240,255), thickness=4)

        # 计算像素距离
        distance = np.linalg.norm([index_tip_x-thumb_tip_x, index_tip_y-thumb_tip_y])
        if(distance < 30):
            print(srceen_mid_mcp_x, srceen_mid_mcp_y)
            pyautogui.rightClick(srceen_mid_mcp_x, srceen_mid_mcp_y)
        distance_str = 'Distance {:.2f}'.format(distance)
        # 在图像上写距离数值，参数依次为：图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
        img_bgr = cv2.putText(img_bgr, distance_str, (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
    img = img_bgr
    return img

Size(width=1920, height=1080)


## 调用摄像头实时画面

In [38]:
# 调用摄像头逐帧实时处理模板
# 不需修改任何代码，只需修改process_frame函数即可
# 同济子豪兄 2021-7-8

# 导入opencv-python
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头
cap = cv2.VideoCapture(0)

# 打开cap
cap.open(0)

# 无限循环，直到break被触发
while cap.isOpened():
    
    # 获取画面
    success, frame = cap.read()
    
    if not success: # 如果获取画面不成功，则退出
        print('获取画面不成功，退出')
        break
    
    ## 逐帧处理
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window',frame)
    
    key_pressed = cv2.waitKey(60) # 每隔多少毫秒毫秒，获取键盘哪个键被按下
    # print('键盘上被按下的键：', key_pressed)

    if key_pressed in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
#     time.sleep(1)
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()

510 722
510 721
512 731
503 758
250 874
656 522
319 869
257 907


In [33]:
print(pyautogui.position())
pyautogui.moveTo(100, 100, duration=0)

Point(x=862, y=700)
